In [1]:
import torch
from torch import nn, optim

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
tensor = torch.tensor([15, 45, 54], device=device)

In [10]:
tensor = tensor.to(torch.float16)


In [13]:
def function01(tensor: torch.Tensor, count_over: str) -> torch.Tensor:
    if count_over == 'columns':
        result = tensor.mean(dim=1)
    if count_over == 'rows':
        result = tensor.mean(dim=0)
    return result
    

In [14]:
function01(tensor, 'rows')

tensor(38., device='cuda:0', dtype=torch.float16)

In [34]:
n_features = 2
n_objects = 300

w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects) / 2

In [21]:
w_true

tensor([0.4359, 0.3109], requires_grad=True)

In [22]:
# grad descent
step = 1e-2

In [35]:
def function02(tensor: torch.Tensor) -> torch.Tensor:
    w = torch.rand(tensor.shape[1], requires_grad=True, dtype=torch.float32)
    return w

In [38]:
def function03(x: torch.Tensor, y: torch.Tensor):
    step=1e-2
    mse = 2
    w = function02(x)
    while (mse > 1):
        y_pred = x @ w
        mse = torch.mean((y - y_pred) ** 2)
        print(mse)

        mse.backward()

        with torch.no_grad():
            w += -w.grad * step
            
        w.grad.zero_()
    return w

In [39]:
function03(X, Y)

tensor(2.3769, grad_fn=<MeanBackward0>)


RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

In [28]:
w = function02(X)
y_pred = X @ w
mse = torch.mean((Y - y_pred) ** 2)

mse.backwards()

with torch.no_grad():
    w += -w.grad * step

w.grad.zero_()

AttributeError: 'Tensor' object has no attribute 'backwards'

In [29]:
mse

tensor(0.2413, grad_fn=<MeanBackward0>)

In [63]:
def function04(x: torch.Tensor, y: torch.Tensor):
    step=1e-2
    mse = 2
    layer = nn.Linear(in_features = x.shape[1], out_features = 1)
    while (mse > 0.3):
        y_pred = layer(x).ravel()
        mse = torch.mean((y - y_pred) ** 2)
        print(mse)

        mse.backward()

        with torch.no_grad():
            layer.weight += -layer.weight.grad * step
            layer.bias += -layer.bias.grad * step
            
        layer.zero_grad()
    return layer

In [64]:
function04(X, Y)

tensor(0.5244, grad_fn=<MeanBackward0>)
tensor(0.5087, grad_fn=<MeanBackward0>)
tensor(0.4940, grad_fn=<MeanBackward0>)
tensor(0.4803, grad_fn=<MeanBackward0>)
tensor(0.4675, grad_fn=<MeanBackward0>)
tensor(0.4556, grad_fn=<MeanBackward0>)
tensor(0.4444, grad_fn=<MeanBackward0>)
tensor(0.4340, grad_fn=<MeanBackward0>)
tensor(0.4243, grad_fn=<MeanBackward0>)
tensor(0.4153, grad_fn=<MeanBackward0>)
tensor(0.4068, grad_fn=<MeanBackward0>)
tensor(0.3988, grad_fn=<MeanBackward0>)
tensor(0.3914, grad_fn=<MeanBackward0>)
tensor(0.3844, grad_fn=<MeanBackward0>)
tensor(0.3779, grad_fn=<MeanBackward0>)
tensor(0.3718, grad_fn=<MeanBackward0>)
tensor(0.3661, grad_fn=<MeanBackward0>)
tensor(0.3607, grad_fn=<MeanBackward0>)
tensor(0.3557, grad_fn=<MeanBackward0>)
tensor(0.3509, grad_fn=<MeanBackward0>)
tensor(0.3465, grad_fn=<MeanBackward0>)
tensor(0.3423, grad_fn=<MeanBackward0>)
tensor(0.3383, grad_fn=<MeanBackward0>)
tensor(0.3346, grad_fn=<MeanBackward0>)
tensor(0.3311, grad_fn=<MeanBackward0>)


Linear(in_features=2, out_features=1, bias=True)

In [4]:
def create_model():
    return nn.Sequential(
        nn.Linear(in_features = 100, out_features = 10),
        nn.ReLU(),
        nn.Linear(in_features = 10, out_features = 1)
    )

In [ ]:
def train(model: nn.Module, data_loader: DataLoader, optimizer: Optimizer, loss_fn):
    model.train()
    total_loss = 0
    for x, y in data_loader:
        optimizer.zero_grad() # zeroing gradients
        output = model(x) # forward pass
        loss = loss_fn(output, y) # count loss
        total_loss += loss.item() # writing current loss
        loss.backward() # backward pass
        print(round(loss.item(), 5)) # print current loss
        optimizer.step() # optimization step
    return total_loss / len(data_loader)


In [1]:
import importlib
task3 = importlib.import_module("3 - ConvNets")
task2 = importlib.import_module("2 - NeuralOptim")
task4 = importlib.import_module("4 - RegularizeConvNet")

%load_ext autoreload
%autoreload 2

In [2]:
task4.get_augmentations(train=True)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    RandAugment(num_ops=2, magnitude=9, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    ToTensor()
    Normalize(mean=[0.49139968 0.48215841 0.44653091], std=[0.24703223 0.24348513 0.26158784])
)

In [11]:
import numpy as np